In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import os
import requests
from dbfread import DBF
import psycopg2
from sqlalchemy import create_engine

In [3]:
###################################################################
## Nome da tabela que será inputada/atualizada no banco de dados ##
###################################################################

# Nome que da tabela para o SQL
nome_tabela = 'tb_sinasc_dn'

# Pasta onde os dados estão armazenados
folder_path = 'C:\Programacao\Python Scripts\dado_tratado'
# Iniciais do arquivo
arquivo = 'SINASC'

# Seleção das variáveis
variaveis_selecionadas = [
                        'CODMUNNASC',
                        'DTNASC',
                        'RACACORMAE',
                        'RACACOR',
                        'DTNASCMAE',
                        'IDADEMAE',
                        'IDANOMAL',
                        'CODANOMAL'
                        ]


def tratamento(df):
    df['id_ano'] = df['DTNASC']%10000
    return df


###################################################################
## Código para carregar, selecionar variáveis e inputar no Banco ##
###################################################################


# Criar uma variável para substituir `pasta` com o caminho do arquivo
def dbf_to_dataframe(pasta): 
    # Lista vazia
    dados = []
    # Ler o DBF e fazer append na lista vazia
    for dado in DBF(pasta, encoding='latin1'):
        dados.append(dado)

    # Converter em dadosframe
    return pd.DataFrame(dados)


# Função para ler o arquivo com base na extensão
def pandas_read(file_path):
    _, file_extension = os.path.splitext(file_path)
    
    if file_extension == '.csv':
        return pd.read_csv(file_path, sep=';', encoding='latin1')
    elif file_extension == '.dbf':
        return dbf_to_dataframe(file_path)  # Substitua a função dbf_to_dataframe pela função correta para ler arquivos .dbf
    else:
        raise ValueError(f"Tipo de arquivo não suportado: {file_extension}")

# DataFrame vazio para armazenar os dados concatenados
concatenated_df = pd.DataFrame()

# Loop nos arquivos no diretório 
for filename in os.listdir(folder_path):
    if filename.startswith(arquivo) and (filename.endswith('.csv') or filename.endswith('.dbf')):
        file_path = os.path.join(folder_path, filename)             # Construção do caminho completo
        df = pandas_read(file_path)                                 # Leitura do arquivo
        
        # Lista de colunas selecionadas (adicione as colunas desejadas aqui)
        df = df[variaveis_selecionadas]
        
        df = tratamento(df)

        treated_df = df.copy()                                      # Aplica a função que retorna o mesmo df
        concatenated_df = pd.concat([concatenated_df, treated_df])

        print(f"Arquivo {filename} carregado e concatenado.")
# Reset index e concatenando em um DataFrame
concatenated_df.reset_index(drop=True, inplace=True)

# Display do DataFrame concatenado
display(concatenated_df)



Arquivo SINASC2016.csv carregado e concatenado.
Arquivo SINASC2017.csv carregado e concatenado.


C:\Users\João Miguel\AppData\Local\Temp\ipykernel_14324\2851265950.py:53: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, sep=';', encoding='latin1')


Arquivo SINASC2018.csv carregado e concatenado.


C:\Users\João Miguel\AppData\Local\Temp\ipykernel_14324\2851265950.py:53: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, sep=';', encoding='latin1')


Arquivo SINASC2019.csv carregado e concatenado.


C:\Users\João Miguel\AppData\Local\Temp\ipykernel_14324\2851265950.py:53: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, sep=';', encoding='latin1')


Arquivo SINASC2020.csv carregado e concatenado.


C:\Users\João Miguel\AppData\Local\Temp\ipykernel_14324\2851265950.py:53: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, sep=';', encoding='latin1')


Arquivo SINASC2021.csv carregado e concatenado.


,CODMUNNASC,DTNASC,RACACORMAE,RACACOR,DTNASCMAE,IDADEMAE,IDANOMAL,CODANOMAL,id_ano
0,110020,18022016,1.0,1.0,10081986.0,29.0,2.0,NaN,2016
1,110020,28012016,4.0,4.0,19032001.0,14.0,2.0,NaN,2016
2,110020,25022016,4.0,4.0,13081985.0,30.0,2.0,NaN,2016
3,110020,20042016,4.0,4.0,15091992.0,23.0,2.0,NaN,2016
4,110020,27042016,4.0,4.0,10071985.0,30.0,2.0,NaN,2016
...,...,...,...,...,...,...,...,...,...
16982654,530010,19092021,1.0,1.0,5031988.0,33.0,2.0,NaN,2021
16982655,530010,30102021,4.0,4.0,10051994.0,27.0,9.0,NaN,2021
16982656,530010,21092021,2.0,2.0,3031995.0,26.0,2.0,NaN,2021
16982657,530010,1122021,NaN,NaN,21121984.0,36.0,2.0,NaN,2021


In [5]:
#####################################
## Configurações do banco de dados ##
#####################################

db_config = {
    'host': 'ls-5ae56aeb2ca5d6cbcd113e0c1aaa14c2a5395de9.c1ozwzg2gdjm.us-east-1.rds.amazonaws.com',
    'database': 'postgres',
    'user': 'postgresql_vs_public',
    'password': '>D,&g-5ZURExX=e-t]8?UXL2j!9B;xtc',
}

# Código que tenta conectar ao banco e carregar o dataframe
try:
    # Conecta ao banco de dados usando o SQLAlchemy
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

    # Carrega o DataFrame para o banco de dados
    concatenated_df.to_sql(nome_tabela, engine, if_exists='replace', index=False) # alterar "replace" no if_exists para "append" e testar com as novas variaveis

    # Caso dê certo
    print("Dados carregados com sucesso!")

    # Caso não dê certo
except (psycopg2.Error, Exception) as e:
    print("Erro ao conectar ao PostgreSQL ou carregar os dados:", e)

Dados carregados com sucesso!
